Testing the whole production using package methods

In [ ]:
from importlib import reload
import helper as hlp

import numpy as np
import numpy.lib.recfunctions

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mpldates
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm, Normalize
%matplotlib inline

import scipy.interpolate as sci
import scipy.optimize as sco
import scipy.integrate as scint
import scipy.stats as scs
import scipy.signal as scsignal

from tqdm import tqdm_notebook as tqdm
import json
import datetime
import pickle
from astropy.time import Time as astrotime
from corner import corner

import sklearn.neighbors as skn
import sklearn.model_selection as skms  # Newer version of grid_search
from sklearn.utils import check_random_state

from corner_hist import corner_hist
from anapymods3.plots.general import (split_axis, get_binmids,
                                      hist_marginalize, dg)
from anapymods3.stats.sampling import rejection_sampling
from anapymods3.general.misc import (fill_dict_defaults,
                                     flatten_list_of_1darrays)

import tdepps.bg_injector as BGInj
import tdepps.bg_rate_injector as BGRateInj
import tdepps.rate_function as RateFunc
import tdepps.llh as LLH
import tdepps.analysis as Analysis
from tdepps.utils import rejection_sampling

secinday = 24. * 60. * 60.

# Load data

Load IC86 data from epinat, which should be the usual IC86-I (2011) PS sample, but pull corrected and OneWeights corrected by number of events generated.

In [ ]:
exp = np.load("data/IC86_I_data.npy")
mc = np.load("data/IC86_I_mc.npy")
# Use the officially stated livetime, not the ones from below
livetime = 332.61

In [ ]:
# Make a global sigma cut (only removes a handful of badly reconstructed evts)
_mc= mc[mc["sigma"] < np.deg2rad(20)]
_exp = exp[exp["sigma"] < np.deg2rad(20)]

# `sample` is used as a wrapper for plotting, where it is sometimes easier to
# have a normal array. Shape is (nevts, nfeatures), each row is a data point
sample = np.vstack((_exp["logE"], _exp["dec"], _exp["sigma"], _exp["ra"])).T
mc_sample = np.vstack((_mc["logE"], _mc["dec"], _mc["sigma"], _mc["ra"])).T

# Test Production Modules

Test if the modules work correctly.
They contain the same code as in the main test notebook, but can be used as classes.
This should simplyfy production.

Currently each submodul only does a very special task:

- `bg_injector`: Samples ("injects") backgorund events for trials
- `bg_rate_injector`: Samples ("injects") the number of BG events to be injected per trial.
- `rate_function`: Describes the time depence of the background rate.
- `llh`: Implements the likelihood function and signal and background PDFs.
- `signal_injector`: Same as `bg_injector` but injecting signal evts from MC.
- `analysis`: Main module pulling it all together, making trials, fitting llhs, provides methods for advanced tasks.

## BG Injector

Injects information for background-like events.

In [ ]:
# Setup for tests in this chapter
n_samples = int(1e5)
rnd_seed = 7353

X_names = data_inj._X_names + ["ra"]
xlabel = ["logE", "dec", "logE", "dec"]
ylabel = ["dec", "sigma", "sigma", "ra"]

axes = [[0, 1], [1, 2], [0,2], [1, 3]]

### Data Resampling

In [ ]:
data_inj = BGInj.DataBGInjector()
data_inj.fit(_exp)
data_sam = data_inj.sample(n_samples, np.random.RandomState(seed=rnd_seed))

# shape (n_samples, n_features) for plotting
_d_sam = np.vstack((data_sam[n] for n in X_names)).T
for i, axis in enumerate(axes):
    fig, (al, ar) = hlp.hist_comp(sample[:, axis], _d_sam[:, axis])
    al.set_xlabel(xlabel[i])
    ar.set_xlabel(xlabel[i])
    al.set_ylabel(ylabel[i])
    ar.set_ylabel(ylabel[i])
    al.set_title("Data")
    ar.set_title("Data sample: {} evts from original Data".format(
        len(_d_sam)))
    plt.show()

### Adaptive Width KDE sampling

In [ ]:
# Assign model from CV, which has already evaluated adaptive kernels.
# Otherwise we would have to reevaluate which takes a long time.
# This should be an official option, to set KDE values for datasets.
with open("data/awKDE_CV/CV10_glob_bw_alpha_EXP_IC86I_CUT_sig.ll.20_" +
          "PARS_diag_True_pass2.pickle", "rb") as f:
    model_selector = pickle.load(f)
    print(model_selector.best_params_)

kde_inj = BGInj.KDEBGInjector()
kde_inj.kde_model = model_selector.best_estimator_

# We could still change the alpha, but the global bandwidth must stay fixed
# kde_inj.kde_model.alpha = 0.3

# Fit doesn't take long because all adaptive kernels are set.
# Note: The original order cannot be changed now [logE, dec, sigma]
bounds = np.array([[None, None], [-np.pi / 2. , np.pi / 2.], [0, None]])
kde_inj.fit(_exp, bounds)

# Sample (bounds are preventing spillover in undefined regions)
kde_sam = kde_inj.sample(n_samples, np.random.RandomState(seed=rnd_seed))
_kde_sam = np.vstack((kde_sam[n] for n in X_names)).T

for i, axis in enumerate(axes):
    fig, (al, ar) = hlp.hist_comp(sample[:, axis], _kde_sam[:, axis])
    al.set_xlabel(xlabel[i])
    ar.set_xlabel(xlabel[i])
    al.set_ylabel(ylabel[i])
    ar.set_ylabel(ylabel[i])
    al.set_title("Data")
    ar.set_title("KDE sample: {} evts".format(len(_kde_sam)))
    plt.show()

### GRBLLH style

In [ ]:
# If False, only sample where data was
# If True sample in global min/max bounding box
minmax = True

mrinj = BGInj.MRichmanBGInjector()
ax0_bins, ax1_bins, ax2_bins = mrinj.fit(_exp, nbins=10, minmax=minmax)
mr_sam = mrinj.sample(n_samples=n_samples,
                      random_state=np.random.RandomState(seed=rnd_seed))
_mr_sam = np.vstack((mr_sam[n] for n in X_names)).T

for i, axis in enumerate(axes):
    fig, (al, ar) = hlp.hist_comp(sample[:, axis], _mr_sam[:, axis])
    al.set_xlabel(xlabel[i])
    ar.set_xlabel(xlabel[i])
    al.set_ylabel(ylabel[i])
    ar.set_ylabel(ylabel[i])
    al.set_title("Data")
    ar.set_title("Pseudo MR sample: {} evts".format(len(_mr_sam)))
    plt.show()

### Pseudo Data (uniform) sampling

In [ ]:
uni_inj = BGInj.UniformBGInjector()
uni_sam = uni_inj.sample(n_samples)
_uni_sam = np.vstack([uni_sam[n] for n in X_names]).T

for i, axis in enumerate(axes):
    fig, (al, ar) = hlp.hist_comp(sample[:, axis], _uni_sam[:, axis])
    al.set_xlabel(xlabel[i])
    ar.set_xlabel(xlabel[i])
    al.set_ylabel(ylabel[i])
    ar.set_ylabel(ylabel[i])
    al.set_title("Data")
    ar.set_title("Pseudo (uniform) sample: {} evts".format(len(_uni_sam)))
    plt.show()

## BG Rate Injector

This module injects times of background like events.

### Injector created from runlist

First step is always to fit a RateFunction to rates from detector runs.
Here we use a Sinus1yrRateFunction with fixed period.

In [ ]:
# First create a rate function. We fix the period to 1 year here
rate_func = RateFunc.Sinus1yrRateFunction()

In [ ]:
# Now parse the rundict and make the fitted injector from that
def filter_runs(run):
    """
    Filter runs as stated in jfeintzig's doc.
    """
    exclude_runs = [120028, 120029, 120030, 120087, 120156, 120157]
    if ((run["good_i3"] == True) & (run["good_it"] == True) &
        (run["run"] not in exclude_runs)):
        return True
    else:
        return False
    
# Let's create an injector using a goodrun list. This creates a run dict
runlist="data/runlists/ic86-i-goodrunlist.json"
runlist_inj = BGRateInj.RunlistBGRateInjector(runlist, filter_runs, rate_func)

# Fit function to exp times to runlist bins
times = exp["timeMJD"]
rate_func = runlist_inj.fit(T=times, x0=None, remove_zero_runs=True)

In [ ]:
# Rebin (donglians proposal)
rates = runlist_inj.rate_rec
start_mjd = rates["start_mjd"]
stop_mjd = rates["stop_mjd"]

tmin, tmax = np.amin(start_mjd), np.amax(stop_mjd)
ntbins = 12
tbins = np.linspace(tmin, tmax, ntbins + 1)

# Get bin idx in which the runs fall
# This is not a 100% correct, because runs may be right over bin edges
idx = np.digitize(stop_mjd, tbins) - 1
rates_per_bin = np.zeros(ntbins, dtype=np.float)

evts_in_run = rates["nevts"]
dts = (stop_mjd - start_mjd) * secinday
for i in range(ntbins):
    rates_per_bin[i] = np.sum(evts_in_run[idx == i]) / np.sum(dts[idx == i])

In [ ]:
# Plot runs (zorder, because errorbar seems to have high zorder for centers)
xerr = 0.5 * (stop_mjd - start_mjd)
yerr = rates["rate_std"]
binmids = 0.5 * (stop_mjd + start_mjd)

plt.errorbar(binmids, rates["rate"], xerr=xerr, yerr=yerr,
             fmt=",", alpha=0.25, zorder=0)
plt.ylim(0, None);

# Plot fit
t = np.linspace(start_mjd[0], stop_mjd[-1], 1000)
y = rate_func(t)
plt.plot(t, y, zorder=5, lw=2, color="C1")

# Plot y shift dashed to see baseline or years average
avg = runlist_inj.best_pars[2]
plt.axhline(avg, 0, 1, color="C1", ls="--", label="", lw=1.5)

plt.xlim(start_mjd[0], stop_mjd[-1])
plt.xlabel("MJD")
plt.ylabel("Rate in Hz")

# Show rebinned (as expected you see nothing new)
m = get_binmids([tbins])[0]
plt.errorbar(m, rates_per_bin, xerr=np.diff(tbins),
             fmt=",", lw=2, color="C2", zorder=3)

# plt.savefig("./data/figs/time_rate_sinus_rebinned.png", dpi=200)
plt.ylim(0, 0.009)
plt.tight_layout()
plt.show()

print("Best fit params:")
for par, name in zip(runlist_inj.best_pars, ["amp", "toff", "base"]):
    print(" {:5} : {:+.3g}".format(name, par))


Sample some trials for a single src and time with the poisson=True keyword to see if we sample correctly for each trial.

Also compare with poisson=False to see if it's working correctly.

In [ ]:
rndgen = np.random.RandomState(7353)

rates = runlist_inj.rate_rec
start_mjd = rates["start_mjd"]

# Pick some random time and time frame
t = np.random.choice(start_mjd, size=1)
trange = np.array([-120, 220])

# This is a list of times per trial
ntrials = int(1e4)
trials = []
for i in range(ntrials):
    trial = runlist_inj.sample(t, trange, poisson=True, random_state=rndgen)
    # Make one array of times, because we have only one src here
    trials.append(flatten_list_of_1darrays(trial))

nevents = np.array(list(map(len, trials)))
print("Sampled total of {:d} events in {:d} trials.".format(
        np.sum(nevents), ntrials))

# Plot poisson distribution of nevents with expectation from integral
expect = runlist_inj.best_estimator_integral(t, trange)
_ = plt.hist(nevents, bins=np.arange(10), normed=True)
plt.axvline(expect, 0, 1, color="C1", ls="--", lw=2, label="expect")
x = np.arange(0, 10)
y = scs.poisson.pmf(x, mu=expect)
_ = plt.plot(x, y, "C1", lw=2, drawstyle="steps-post")
plt.legend()
plt.show()

# Now the same for possion=False as a crosscheck
trials = []
for i in range(ntrials):
    trial = runlist_inj.sample(t, trange, poisson=False, random_state=rndgen)
    # Make one array of times, because we have only one src here
    trials.append(flatten_list_of_1darrays(trial))

nevents = np.array(list(map(len, trials)))
print("Sampled total of {:d} events in {:d} trials.".format(
        np.sum(nevents), ntrials))

# Plot poisson distribution of nevents with expectation from integral, here
# for comparison to the previous case only
expect = runlist_inj.best_estimator_integral(t, trange)
_ = plt.hist(nevents, bins=np.arange(10), normed=True)
plt.axvline(expect, 0, 1, color="C1", ls="--", lw=2, alpha=0.5,
            label="expect")
plt.axvline(np.round(expect), 0, 1, color="C1", ls="--", lw=2,
            label="round expect")
x = np.arange(0, 10)
y = scs.poisson.pmf(x, mu=expect)
_ = plt.plot(x, y, "C1", lw=2, drawstyle="steps-post")
plt.legend()
plt.show()

Now we do the same, but with multiple sources.
Each src gets a larger time window, so the expectation gets higher and we can compare different poisson distributions at once.

In [ ]:
rndgen = np.random.RandomState(7353)

rates = runlist_inj.rate_rec
start_mjd = rates["start_mjd"]

# Pick random times and make increasing time frames per source
nsrcs = 3
t = np.random.choice(start_mjd, size=nsrcs)
trange = np.vstack((np.repeat([-100], nsrcs),
                    500 * np.arange(1, 3 * nsrcs + 1, 3))).T

# This is a list of times per trial
ntrials = int(1e4)
trials = []
for i in range(ntrials):
    trial = runlist_inj.sample(t, trange, poisson=True, random_state=rndgen)
    # Make one array of times, because we have only one src here
    trials.append(trial)

# The format of `trials` is list(array_src1, array_src2, ...) for each trial.
# We want the number of events sampled per src per trial
nevents = []
for i in range(nsrcs):
    nevents.append([len(trial[i]) for trial in trials])
    print("Sampled {:d} events in {:d} trials for src {:d}.".format(
          np.sum(nevents[i]), ntrials, i))

# Plot poisson distributions of nevents with expectations from integrals
expect = runlist_inj.best_estimator_integral(t, trange)
colors = ["C0", "C1", "C3"]
for i in range(nsrcs):
    _ = plt.hist(nevents[i], bins=np.arange(np.amax(nevents)), normed=True,
                 color=colors[i], alpha=.25)
    plt.axvline(expect[i], 0, 1, ls="--", lw=2, label="mu src {}".format(i),
                color=colors[i])
    x = np.arange(0, np.amax(nevents))
    y = scs.poisson.pmf(x, mu=expect[i])
    _ = plt.plot(x, y, lw=2, drawstyle="steps-post", color=colors[i])

plt.legend()
plt.show()

Now we want to look at the actual sampled times in each trial.
First we sample a single in a small timeframe.
It should be approximately uniformly distributed, respectively not to distinguish by eye from a constant PDF, because the sine is way to broad to be resolved on scuh a small time scale.
We also show the bg and signal pdf for comparison.

In [ ]:
rndgen = np.random.RandomState(7353)

# First the small time frame
# Arbitrary start date from data
nsrcs = 1
t0 = np.random.choice(start_mjd, size=nsrcs)
t0_sec = t0 * secinday

# dt from t0 in seconds, clip at 4 sigma
dt = 200
nsig = 4.

# Make t values for plotting in MJD around t0
clip = np.clip(dt, 2, 30) * nsig
trange = np.array([-clip, dt + clip]).reshape(nsrcs, 2)
ntrials = int(1e4)

# Sample times for each trial and flatten to single array with all trials
trials = []
for i in range(ntrials):
    trials += runlist_inj.sample(t0, trange, poisson=True,
                                 random_state=rndgen)
trials = flatten_list_of_1darrays(trials)

# Plot them in together with the PDFs
def time_bg_pdf(t, t0, a, b):
    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
  
    pdf = np.zeros_like(_t, dtype=np.float)
    uni = (_t >= a) & (_t <= b)
    pdf[uni] = 1. / (b - a)
    return pdf

def time_sig_pdf(t, t0, dt, nsig=4):
    if dt < 0:
        raise ValueError("dt must not be negative.")

    # Normalize relative to t0 in seconds (first multiply avoids rounding?)
    _t = t * secinday - t0 * secinday
    
    # Constrain sig_t to [2, 30]s regardless of uniform time window
    sig_t = np.clip(dt, 2, 30)
    sig_t_clip = nsig * sig_t
    gaus_norm = (np.sqrt(2 * np.pi) * sig_t)
    
    # Split in def regions gaus rising, uniform, gaus falling and zero
    gr = (_t < 0) & (_t >= -sig_t_clip)
    gf = (_t > dt) & (_t <= dt + sig_t_clip)
    uni = (_t >= 0) & (_t <= dt)
    
    pdf = np.zeros_like(t, dtype=np.float)
    pdf[gr] = scs.norm.pdf(_t[gr], loc=0, scale=sig_t)
    pdf[gf] = scs.norm.pdf(_t[gf], loc=dt, scale=sig_t)
    # Connect smoothly with the gaussians
    pdf[uni] = 1. / gaus_norm
    
    # Normalize whole distribtuion
    dcdf = (scs.norm.cdf(dt + sig_t_clip, loc=dt, scale=sig_t) -
            scs.norm.cdf(-sig_t_clip, loc=0., scale=sig_t))
    norm = dcdf + dt / gaus_norm
    
    return pdf / norm


# Plot the pdfs
t = np.linspace(t0_sec + trange[:, 0], t0_sec + trange[:, 1], 200) / secinday
bg_pdf = time_bg_pdf(t, t0, -clip, dt + clip)
sig_pdf = time_sig_pdf(t, t0, dt, nsig)

# Plot in normalized time
_t = t * secinday - t0 * secinday
plt.plot(_t, bg_pdf, "C0-")
plt.plot(_t, sig_pdf, "C1-")
plt.axvline(dt, 0, 1, color="C3", ls="--")
plt.axvline(0, 0, 1, color="C2", ls="--")

# Plot injected events from all trials, relative times
times = (trials - t0) * secinday
_ = plt.hist(times, bins=50, normed=True, color=dg, alpha=.25)

plt.xlabel("Time relative to t0 in sec")
plt.ylim(0, None);
plt.tight_layout()

# plt.savefig("./data/figs/bg_events_time_sampled_narrow.png", dpi=200)

plt.show()

In [ ]:
rndgen = np.random.RandomState(7353)

# Now the really large time frame, over the whole time range
t0 = start_mjd[0]
t0_sec = t0 * secinday

# Maximum dt over all runs
dt = (stop_mjd[-1] - start_mjd[0]) * secinday
nsig = 4.

# Make t values for plotting in MJD around t0
clip = np.clip(dt, 2, 30) * nsig
trange = [-clip, dt + clip]
ntrials = 100  # More trials mean smaller errors, better see the sinus shape 

# Sample times
trials = []
for i in range(ntrials):
    trials += runlist_inj.sample(t0, trange, poisson=True,
                                 random_state=rndgen)
trials = flatten_list_of_1darrays(trials)

# We choose the same style as in the intial rate plot further above
h, b = np.histogram(trials, bins=1081)
m = get_binmids([b])[0]
scale = np.diff(b) * secinday * ntrials
yerr = np.sqrt(h) / scale
h = h / scale

plt.errorbar(m, h, yerr=yerr, fmt=",")

# Plot normalized rate function to compare
t = np.linspace(start_mjd[0], stop_mjd[-1], 100)
r = runlist_inj.best_estimator(t)
plt.plot(t, r, lw=2, zorder=5)
plt.axhline(runlist_inj.best_pars[2], 0, 1, color="C1",
            ls="--", label="", zorder=5)

plt.xlim(start_mjd[0], stop_mjd[-1])
plt.ylim(0.004, 0.006)
plt.tight_layout()

# plt.savefig("./data/figs/bg_events_time_sampled_wide.png", dpi=200)

plt.show()

## Utils - rejection_sampler

Test the utils.py rejection sampler.
We generate trials for multiple sources at once and check how fast this is.
Currently the method just loops over sources, rejection sampling for each interval, but it seems fast enough.

A short note on the test below:
We make nsrcs, each with ordered center times and increasing time windows.
Then we sample an increasing number of samples per source.
This is the same as a single trial.

In the histogram we expect 2 things:

1. If the time windows are smaller than 1 day, which is the bin size, then we just get a nice lineraly increaing bin content (triangle shaped, with hard cut at the right edge).
2. If the time windows increase, we get spillover resulting in a way more spread distribution. Also the time windows are only widened to to the right, so the spillover occurs only to the right edges. When the time windows are really large, we even begin to see the underlying oscillation of the sinusodial test function we generate samples from.

In [ ]:
def sample_test_sin(t):
    """Simple sinus, similar to fitted rate function"""
    return 0.001 * np.sin(2 * np.pi / 365. * (t - 50000)) + 0.005

# Make some srcs and incresing time windows
nsrcs = 100
t = np.arange(0, nsrcs) + 50000
scaler = 2 * secinday  # Time window scaler: Increase to see spillover
dts = np.vstack((np.zeros(nsrcs), scaler * np.arange(1, nsrcs + 1))).T
dts = t.reshape(nsrcs, 1) + dts / secinday

# Sample increasing number of events in time windows
n_samples = 100 * np.arange(1, nsrcs + 1)
sample = rejection_sampling(sample_test_sin, dts,
                            n_samples=n_samples, random_state=3537)

flatsam = flatten_list_of_1darrays(sample)

# If the time windows are larger than one day (the binning) we get spillover
_ = plt.hist(flatsam, bins=nsrcs)
plt.show()

In [ ]:
# Let's look at the distribution in the largest time window.
# It should be sinusodial
_ = plt.hist(sample[-1], bins=20, normed=True)
# Plot sampled function as comparison
t = np.linspace(dts[-1, 0],dts[-1, 1], 100)
intgrl = scint.quad(sample_test_sin, dts[-1, 0],dts[-1, 1])[0]
y = sample_test_sin(t) / intgrl
plt.plot(t, y)
plt.show()

In [ ]:
%%timeit
# Quickly check how fast we are. Set nsrcs to 100 above for many srcs
rejection_sampling(sample_test_sin, dts, n_samples=n_samples,
                   random_state=3537)

## BG Rate Function

Test if fit, sample and integral works, with a simple example.
First for only a single source.

### SinusRateFunction

In [ ]:
# Define parameters for the test function
period_days = 300.
b = 2 * np.pi / period_days  # Period in 1/MJD
c = 0  # t-Offset in MJD
d = 1  # Rate offset in Hz = 1 evt/sec is average -> 86400 evts/day
a = d / 2.  # Amplitude in Hz = +- 0.5 evts / per second
pars = np.array([a, b, c, d])

sinfun = RateFunc.SinusRateFunction()

# Plot function
t0, t1 = c, c + period_days
t = np.linspace(0, t1, 200)  # In MJD days
y = sinfun.fun(t, pars)

_ = plt.plot(t, y, lw=2, label="fun")

# Plot integral
intgrl = np.zeros_like(t)
for i, ti in enumerate(t):
    intgrl[i] = sinfun.integral(t=t0, trange=[t0, ti*secinday], pars=pars)
    
# Scale integral, we expect 24*3600=86400 evts/day * (period_days days)
print("Expect   : ", secinday * t1)
print("Integral : ", intgrl[-1])
_ = plt.plot(t, intgrl / 1e7, lw=2, label="integral/1e7")

# Sample from whole range and scale normed hist with time scale to match rate
nsam = [int(1e4),]
trange = np.array([[t0, t1],]) * secinday
sam = sinfun.sample(t=t0, trange=trange,
                    pars=pars, n_samples=nsam)
h, b = np.histogram(sam, range=[t0, t1], bins=50, density=True)
m = get_binmids([b])[0]
_ = plt.hist(m, bins=b, weights=h * (t1 - t0), color="C0",
             alpha=0.5, label="sampled")

# Finally fit the sampled points again
runtime = (t1 - t0)
p0 = None  # Test default args
bf_pars = sinfun.fit(t=m, rate=h * (t1 - t0), rate_std=None, p0=p0)
yfit = sinfun.fun(t, bf_pars)
_ = plt.plot(t, yfit, lw=2, color="C3", ls="--", label="fitted")
p0 = sinfun._get_default_seed(t=m, rate=h * (t1 - t0),
                              rate_std=np.ones_like(m))
yseed = sinfun.fun(t, p0)
_ = plt.plot(t, yseed, lw=2, color="C3", ls="-", alpha=0.3,
             label="default seed")

plt.xlabel("time in MJD")
plt.ylabel("rate in Hz")
_ = plt.ylim(0, None)
plt.legend()
plt.tight_layout()

# plt.savefig("data/figs/rate_function_test.png", dpi=200)

plt.show()

### Sinus1yrRateFunction

The same as above, but now with fixed period of 1 year.

In [ ]:
# Define parameters for the test function
period_days = 365.25
c = 0  # t-Offset in MJD
d = 1  # Rate offset in Hz = 1 evt/sec is average -> 86400 evts/day
a = d / 2.  # Amplitude in Hz = +- 0.5 evts / per second
pars = np.array([a, c, d])

sinfun = RateFunc.Sinus1yrRateFunction()

# Plot function
t0, t1 = c, c + period_days
t = np.linspace(0, t1, 200)  # In MJD days
y = sinfun.fun(t, pars)

_ = plt.plot(t, y, lw=2, label="fun")

# Plot integral
intgrl = np.zeros_like(t)
for i, ti in enumerate(t):
    intgrl[i] = sinfun.integral(t=t0, trange=[t0, ti*secinday], pars=pars)
    
# Scale integral, we expect 24*3600=86400 evts/day * (period_days days)
print("Expect   : ", secinday * t1)
print("Integral : ", intgrl[-1])
_ = plt.plot(t, intgrl / 1e7, lw=2, label="integral/1e7")

# Sample from whole range and scale normed hist with time scale to match rate
nsam = int(1e4)
sam = sinfun.sample(t=t0, trange=[t0, t1*secinday], pars=pars, n_samples=nsam)
h, b = np.histogram(sam, range=[t0, t1], bins=50, density=True)
m = get_binmids([b])[0]
_ = plt.hist(m, bins=b, weights=h * (t1 - t0), color="C0",
             alpha=0.5, label="sampled")

# Finally fit the sampled points again
runtime = (t1 - t0)
p0 = None  # Test default args
bf_pars = sinfun.fit(t=m, rate=h * (t1 - t0), rate_std=None, p0=p0)
yfit = sinfun.fun(t, bf_pars)
_ = plt.plot(t, yfit, lw=2, color="C3", ls="--", label="fitted")
p0 = sinfun._get_default_seed(t=m, rate=h * (t1 - t0),
                              rate_std=np.ones_like(m))
yseed = sinfun.fun(t, p0)
_ = plt.plot(t, yseed, lw=2, color="C3", ls="-", alpha=0.3,
             label="default seed")

plt.xlabel("time in MJD")
plt.ylabel("rate in Hz")
_ = plt.ylim(0, None)
plt.legend()
plt.tight_layout()

# plt.savefig("data/figs/rate_function_test.png", dpi=200)

plt.show()

### ConstantRateFunction

Use constant rate function but leave the sinus to see how the fit behaves.
Otherwise it would be boring to just see 3 flat lines over another.

In [ ]:
# Define sinus parameters
period_days = 365.25
c = 0  # t-Offset in MJD
d = 1  # Rate offset in Hz = 1 evt/sec is average -> 86400 evts/day
a = d / 2.  # Amplitude in Hz = +- 0.5 evts / per second
sinpars = np.array([a, c, d])

# Same for the constant function.
constpars = (d,)

sinfun = RateFunc.Sinus1yrRateFunction()
constfun = RateFunc.ConstantRateFunction()

# Plot sinus and constant function
t0, t1 = c, c + period_days
t = np.linspace(0, t1, 200)  # In MJD days
y = sinfun.fun(t, sinpars)
yc = constfun.fun(t, constpars)

_ = plt.plot(t, y, color="C0", lw=2, ls="--")
_ = plt.plot(t, yc, lw=2, label="fun")

# Plot integral
intgrl = np.zeros_like(t)
for i, ti in enumerate(t):
    intgrl[i] = constfun.integral(t=t0, trange=[t0, ti*secinday],
                                  pars=constpars)
    
# Scale integral, we expect 24*3600=86400 evts/day * (period_days days)
print("Expect   : ", secinday * t1)
print("Integral : ", intgrl[-1])
_ = plt.plot(t, intgrl / 1e7, lw=2, label="integral/1e7")

# Sample from whole range and scale normed hist with time scale to match rate
nsam = int(1e4)
sam = sinfun.sample(t=t0, trange=[t0, t1*secinday], pars=sinpars,
                      n_samples=nsam)
h, b = np.histogram(sam, range=[t0, t1], bins=50, density=True)
m = get_binmids([b])[0]
_ = plt.hist(m, bins=b, weights=h * (t1 - t0), color="C0",
             alpha=0.5, label="sampled")

# Finally fit the sampled points again
runtime = (t1 - t0)
p0 = None  # Test default args
bf_pars = constfun.fit(t=m, rate=h * (t1 - t0), rate_std=None, p0=p0)
yfit = constfun.fun(t, bf_pars)
_ = plt.plot(t, yfit, lw=2, color="C3", ls="--", label="fitted")
p0 = constfun._get_default_seed(t=m, rate=h * (t1 - t0),
                                rate_std=np.ones_like(m))
yseed = constfun.fun(t, p0)
_ = plt.plot(t, yseed, lw=2, color="C3", ls="-", alpha=0.3,
             label="default seed")

plt.xlabel("time in MJD")
plt.ylabel("rate in Hz")
_ = plt.ylim(0, None)
plt.legend()
plt.tight_layout()

# plt.savefig("data/figs/rate_function_test.png", dpi=200)

plt.show()

## LLH

Test the LLH module.

It contains all functions for a specific LLH we want to use in our analysis.
Currently GRBLLH is implemented.

In [ ]:
sin_dec_bins = np.linspace(-1, 1, 50)

min_logE = 1  #  min(np.amin(_exp["logE"]), np.amin(mc["logE"]))
max_logE = 10 #  max(np.amax(_exp["logE"]), np.amax(mc["logE"]))
logE_bins = np.linspace(min_logE, max_logE, 40)

spatial_pdf_args = {"bins": sin_dec_bins, "k": 3, "kent": True}

energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "col", "interpol_log": False}

time_pdf_args = {"nsig": 4., "sigma_t_min": 2., "sigma_t_max": 30.}

grbllh = LLH.GRBLLH(X=_exp, MC=mc, srcs=None,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

### Time PDF Ratio

Reproduce the paper plot.

Note that we get the PDFs for all srcs at once.
Their times are just all the same here.

In [ ]:
# Make a plot with ratios for different time windows as in the paper
# dt from t0 in seconds, clip at 4 sigma
dts = [[-1, 5], [-5, 50], [-20, 200]]
nsrcs = len(dts)
nsig = 4

# Arbitrary start date from data
t0 = np.repeat(np.random.choice(exp["timeMJD"], size=1),
               repeats=nsrcs).reshape(nsrcs, 1)
t0_sec = t0[0] * secinday  # Only single number needed, t0s are all equal

# Make t values for plotting in MJD around t0, to fit all in one plot
max_dt, min_dt = np.amax(dts), np.amin(dts)
dt_tot = max_dt - min_dt
clip = np.clip(dt_tot, 2, 30) * nsig
plt_range = np.array([min_dt - clip, max_dt + clip])
t = np.linspace(t0_sec + 1.2 * plt_range[0],
                t0_sec + 1.2 * plt_range[1], 1000) / secinday
_t = t * secinday - t0 * secinday

# Mark event time
plt.axvline(0, 0, 1, c="k", ls="--", lw=2, alpha=0.8)

# Get all at once
SoB = grbllh._soverb_time(t=t, src_t=t0, dt=dts)

colors = ["C0", "C3", "C2"]
for i in range(len(SoB)):
    # Plot seperately to give colors and labels
    plt.plot(_t[i], SoB[i], lw=2, c=colors[i],
             label=r"$T_\mathrm{{uni}}$: {:>3d}s, {:>3d}s".format(*dts[i]))
    # Fill uniform part, might look nicely
    # fbtw = (_t > 0) & (_t < dt)
    # plt.fill_between(_t[fbtw], 0, SoB[fbtw], color="C7", alpha=0.1)

# Plot stacked
weights = np.ones(nsrcs).reshape(nsrcs, 1) / nsrcs
stacked = np.sum(SoB * weights, axis=0) 
plt.plot(_t[0], stacked, ls="--", c="k", lw=2, label="stacked", alpha=0.8)
    
# Make it look like the paper plot, but with slightly extended borders
plt.xlim(1.2 * plt_range)
plt.ylim(0, np.amax(SoB) * 1.05)
plt.xlabel("t - t0 in sec")
plt.ylabel("S / B")
plt.legend(loc="upper right")
plt.grid(ls="--", lw=1)

# plt.savefig("./data/figs/time_pdf_ratio.png", dpi=200)

plt.show()

Get the injection time window.
This is needed for the injector, so only events in regions with non-zero PDF are injected.

In [ ]:
grbllh.get_injection_trange(t0, dts)

In [ ]:
# Compare manually
dts = np.array([[-1, 5], [-5, 50], [-20, 200]], dtype=np.float)
nsig, sig_min, sig_max = time_pdf_args.values()  # Beware if order is wrong :P
clip = np.clip(np.diff(dts, axis=1), sig_min, sig_max) * nsig
dts[:, 0] -= clip.reshape(len(dts))  # Same as flatten()
dts[:, 1] += clip.flatten()

dts

### Spatial background spline

This is the same technique as used in skylab, but with an extra step of adding the outermost bin edges to the spline gridpoints.
This way, the spline behaves reasonable at the edges and doesn't overshoot.

We could extend this by using the KDE integrated over every variable and then fitting a spline to that.
Or we could sample from the KDE and bin finely and fit a splien again.

For now we leave only the option to use data directly.
The spline fit is depending on the binning anyway.
Only the finely binned KDE version could resolve that issue.

In [ ]:
sin_dec = np.linspace(-1.05, 1.05, 200)
y = np.exp(grbllh._spatial_bg_spl(sin_dec))
_ = plt.hist(np.sin(_exp["dec"]), bins=50, normed=True)
plt.plot(sin_dec, y, lw=2)

### Spatial background pdf

Should be identical to calling the spline directly, except that the BG PDF is normalized to the whole sphere.
So we multiply the values by 2pi to account for that.

Here we see the difference to just calling the spline directly: The PDF is zero outside the definition range, the spline extrapolated.

In [ ]:
_ = plt.hist(np.sin(_exp["dec"]), bins=grbllh.spatial_pdf_args["bins"],
             normed=True)
sin_dec = np.linspace(-1.05, 1.05, 200)
y = 2 * np.pi * grbllh._pdf_spatial_background(ev_sin_dec=sin_dec)
plt.plot(sin_dec, y, lw=2)

### Spatial signal PDF

Compare signal and BG pdf.

First we create multiple sources and a single event and scan the event PDF by moving the event along the declination axis.
All PDFs have the height, because the same sigma is used.

Note that BG is here usually very small compared to the signal, because we sample the ev positions within 1 sigma around the source.

In [ ]:
LOG = False

nsrcs = 4
# Choose the event sigma from data
ev_sigma = np.random.choice(_exp["sigma"], size=1)

# Make nsrcs, same ra, but different dec. decs are distributed uniformly in
# the range of the largest sigma from the events (for illustration only)
src_dec = np.random.uniform(-ev_sigma, ev_sigma, size=nsrcs)
src_ra = np.ones_like(src_dec) * np.pi
plt_rnge = [np.amin(src_dec) - ev_sigma, np.amax(src_dec) + ev_sigma]

# Scan signal PDF for event declination
ev_dec = np.sin(np.linspace(plt_rnge[0], plt_rnge[1], 200))
ev_sin_dec = np.sin(ev_dec)
ev_ra = src_ra[0] * np.ones_like(ev_sin_dec)
ev_sig = np.ones_like(ev_sin_dec) * ev_sigma

# y has shape (nsrcs, nevts), where nevts are the ev_sin_dec values here (scan)
y = grbllh._pdf_spatial_signal(src_ra, src_dec, ev_ra, ev_sin_dec, ev_sig)

if LOG:
    y = np.log10(y)

plt.plot(ev_dec, y.T, lw=2)
plt.vlines(src_dec, 0, np.amax(y), color="C7", linestyles="--", lw=2,
           label="srcs pos")

# Plot BG PDF to compare
bg = grbllh._pdf_spatial_background(ev_sin_dec=ev_sin_dec)
plt.plot(ev_dec, bg, lw=2, label="BG")

plt.xlim(*plt_rnge)
if LOG:
    plt.ylim(1e-5, 1.1 * np.amax(y))
else:
    plt.ylim(0, 1.1 * np.amax(y))
    
    
plt.xlabel("dec")
plt.ylabel("PDF per src")
plt.legend()
    
plt.tight_layout()

Here we use multiple events with different sigmas and scan again in declination by moving a single possible src position.
We get different heights, because of the different sigmas.

The PDFs each peak where the event position is.
If we had a single source, we would just read off the values at that position.

In [ ]:
LOG = True
# Make nevts, same ra, but different dec. sigmas chosen from data
nevts = 4
ev_sigma = np.random.choice(_exp["sigma"], size=nevts)
# Sample some evt decs uniformly around the horizon with spread of the largest 
# sigma to get some variation
max_sig = np.amax(ev_sigma)
ev_dec = np.random.uniform(-max_sig, max_sig, size=nevts)
ev_sin_dec = np.sin(ev_dec)
ev_ra = np.ones_like(ev_dec) * np.pi

# Plot margin PDF scanned for each src position for each event position
src_dec = np.linspace(-2. * max_sig, 2 * max_sig, 200)
src_ra = ev_ra[0] * np.ones_like(src_dec)

# This has shape (nsrcs, nevts)
y = grbllh._pdf_spatial_signal(src_ra, src_dec, ev_ra, ev_sin_dec, ev_sigma)

if LOG:
    y = np.log10(y)

plt.plot(src_dec, y, lw=2)

plt.vlines(ev_dec, 0, np.amax(y) * 1.1, color="C7",
           linestyles="--", label="evts pos")

# Plot BG PDF to compare
bg = grbllh._pdf_spatial_background(ev_sin_dec=np.sin(src_dec))
plt.plot(src_dec, bg, lw=2, label="BG")

plt.xlim(src_dec[[0, -1]])
if LOG:
    plt.ylim(1e-5, 1.1 * np.amax(y))
else:
    plt.ylim(0, 1.1 * np.amax(y))
    
plt.legend()
plt.tight_layout()

### Spatial PDF ratio

In [ ]:
def plot_dec_vs_signal(S, ev_dec, src_ra, src_dec, weights, ax=None):
    if ax is None:
        _, ax = plt.subplots(1, 1)
    # Plot signal per source for each event
    for i, (sra, sdec) in enumerate(zip(src_ra, src_dec)):
        ax.plot(np.rad2deg(ev_dec), S[i], ls="-")
        ax.plot(np.rad2deg(sdec), -10, "k|")

    # Simulate a simple stacking, one weight per source
    ax.plot(np.rad2deg(ev_dec), np.sum(weights * S, axis=0) / np.sum(weights),
             ls="--", c=dg, label="stacked")

    ax.set_xlim([-1 + smin, smax + 1])
    ax.set_xlabel("DEC in °")
    ax.set_ylabel("Signal pdf")
    ax.legend(loc="upper right")
    return ax

We make 4 plots to test everything:

1. [Top left] We place densely packed srcs at the declination range and scan the PDFs by varying the event declinations.
   Sigma is fixed to 1 for illustration.
   We expect just a row of gaussians along the dec range.
   The stacked signal is the weighted sum of all signal contributions at a single event dec position.
   
2. [Bottom left] We plot just the background PDF and its inverse for the dec range.
   The inverse PDF is what modulates the signal PDF.
   
3. [Top right] This modulation can be seen in this plot.
   It is basically the same as the first one, but now it's signal over background.
   So the signal peaks are modulated with the inverse BG PDF.
   
4. [Bottom right] This is the same plot as the third one, but this time we use the real data declination values instead of nicely spaced ones.
   The effect is the same but not reall visible, because each event has a different sigma, so the PDFs all have different heights and widths.
   It becomes more similar when using an 1° sigma for all events (just comment that line in).

In [ ]:
# Make srcs across the dec range. The hull of SoB should be shaped like the
# 1/(sinDec BG distribtuion). With a single source we couldn't see that,
# because it drops to zero far from the src position
smin, smax, step = -90, +90, 10
src_ra = np.deg2rad(np.arange(smin, smax + step, step))
src_dec = np.deg2rad(np.arange(smin, smax + step, step))

# Scan in dec by varying the evts dec
ev_ra = np.deg2rad(np.linspace(smin, smax, 1000))
ev_dec = np.deg2rad(np.linspace(smin, smax, 1000))
ev_sin_dec = np.sin(ev_dec)
ev_sig = np.deg2rad(np.ones_like(ev_ra))

# Some pseudo weights to simulate stacking
weights = np.arange(1, len(src_dec) + 1)[:, np.newaxis]

fig, ((axtl, axtr), (axbl, axbr)) = plt.subplots(2, 2, figsize=(12, 10))

# Signal only
S = grbllh._pdf_spatial_signal(src_ra, src_dec, ev_ra, ev_sin_dec, ev_sig)
_ = plot_dec_vs_signal(S, ev_dec, src_ra, src_dec, weights, ax=axtl)
axtl.set_xlim(-90, 90)

# Background only
bins = grbllh.spatial_pdf_args["bins"]
h, b = np.histogram(np.sin(_exp["dec"]), bins=bins, density=True)
m = 0.5 * (b[:-1] + b[1:])
_ = axbl.hist(m, bins=bins, weights=h / 2 / np.pi, alpha=0.5)
_sin_dec = np.linspace(-1, 1, 1000)
bg_pdf = grbllh._pdf_spatial_background(_sin_dec)
axbl.plot(_sin_dec, bg_pdf, lw=2, label="pdf")
axbl.set_ylim(0, 0.2)
# 1 / BG PDF on second axis
axbl2 = axbl.twinx()
axbl2.plot(_sin_dec, 1. / bg_pdf, c="C2", lw=2, ls="--", label="1/pdf")
axbl2.set_ylim(0, (1 / bg_pdf).max())
axbl.set_xlabel("sinus DEC")
axbl.set_xlim(-1, 1)
axbl.legend(loc="upper left")
axbl2.legend(loc="upper center")

# SoB on example + BG PDF
SoB = grbllh._soverb_spatial(src_ra, src_dec, ev_ra, ev_sin_dec, ev_sig)
weights = np.arange(1, len(src_dec) + 1)[:, np.newaxis]
_ = plot_dec_vs_signal(SoB, ev_dec, src_ra, src_dec, weights, ax=axtr)
axtr.plot(np.rad2deg(np.arcsin(_sin_dec)), bg_pdf, lw=3, label="BG pdf", c=dg)
axtr.set_xlim(-90, 90)
axtr.set_yscale("log")
axtr.set_ylim(np.amin(bg_pdf), 1e5)
axtr.legend(loc="upper left")

# Now with the real data. Sort first in dec to show with nice lines + BG PDF
idx = np.argsort(exp["dec"])
ev_ra = exp["ra"][idx]
ev_dec = exp["dec"][idx]
ev_sin_dec = np.sin(ev_dec)
ev_sig = exp["sigma"][idx]
# Comment in to match the simple example (all events have sigma 1°)
# ev_sig = np.deg2rad(np.ones_like(ev_ra))
SoB = grbllh._soverb_spatial(src_ra, src_dec, ev_ra, ev_sin_dec, ev_sig)

_ = plot_dec_vs_signal(SoB, ev_dec, src_ra, src_dec, weights, ax=axbr)
axbr.plot(np.rad2deg(np.arcsin(_sin_dec)), bg_pdf,
          lw=3, label="BG pdf", c="C0")
axbr.set_yscale("log")
axbr.set_ylim(np.amin(bg_pdf), 1e5)
axbr.legend(loc="upper left")

plt.show()

### Energy ratio spline

This is the creation of the signal over background ratio for the energy PDF.
It is resolved in sinDec and logE to account for different positions on the sky and energies.

Missing values, where no data or MC is present is filled with interpolation values, conttrolled by the "fillval" option.

In [ ]:
fig, (al, ar) = plt.subplots(1, 2, figsize=(14,5))

energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "col", "interpol_log": False}
grbllh = LLH.GRBLLH(X=_exp, MC=mc, srcs=None,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

# Ratio spline with 'col' filling
x = np.linspace(-1.1, 1.1, num=1000 + 1)
y = np.linspace(0.5, 10.5, num=1000 + 1)
XX, YY = np.meshgrid(x, y)
xx, yy = map(np.ravel, [XX, YY])
gpts = np.vstack((xx, yy)).T
zz = np.exp(grbllh._energy_spl(gpts))
ZZ = zz.reshape(XX.shape)
# Plotting with hist creates strange effects... Use pcolormesh instead
img = al.pcolormesh(XX, YY, ZZ, norm=LogNorm(), cmap="coolwarm",
                    vmin=1e-3, vmax=1e3)
al.set_title("Spline interpolation: 'col'")
plt.colorbar(ax=al, mappable=img)

# With 'minmax' filling. Note: The small values in the lower row are due to
# plotting in log. We interpolate in linear space, so in log, the jump is
# very steep for small values.
energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "minmax", "interpol_log": False}
grbllh = LLH.GRBLLH(X=_exp, MC=mc, srcs=None,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

zz = np.exp(grbllh._energy_spl(gpts))
ZZ = zz.reshape(XX.shape)
img = ar.pcolormesh(XX, YY, ZZ, norm=LogNorm(), cmap="coolwarm",
                    vmin=1e-3, vmax=1e3)
ar.set_title("Spline interpolation: 'minmax'")
plt.colorbar(ax=ar, mappable=img)

fig.tight_layout()

### Energy PDF ratio

Here we see again the difference to the direct spline evaluation.
The ratio function set's values outside to zero probability.

In [ ]:
fig, (al, ar) = plt.subplots(1, 2, figsize=(14,5))

energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "col", "interpol_log": True}
grbllh = LLH.GRBLLH(X=_exp, MC=mc, srcs=None,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

# Ratio spline with 'col' filling
x = np.linspace(-1.1, 1.1, num=1000 + 1)
y = np.linspace(0.5, 10.5, num=1000 + 1)
XX, YY = np.meshgrid(x, y)
xx, yy = map(np.ravel, [XX, YY])
gpts = np.vstack((xx, yy)).T
zz = grbllh._soverb_energy(xx, yy)
ZZ = zz.reshape(XX.shape)
# Plotting with hist creates strange effects... Use pcolormesh instead
img = al.pcolormesh(XX, YY, ZZ, norm=LogNorm(), cmap="coolwarm",
                    vmin=1e-3, vmax=1e3)
al.set_title("Spline interpolation: 'col'")
plt.colorbar(ax=al, mappable=img)

# With 'minmax' filling. Note: The small values in the lower row are due to
# plotting in log. We interpolate in linear space, so in log, the jump is
# very steep for small values.
energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "minmax", "interpol_log": True}
grbllh = LLH.GRBLLH(X=_exp, MC=mc, srcs=None,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

zz = grbllh._soverb_energy(xx, yy)
ZZ = zz.reshape(XX.shape)
img = ar.pcolormesh(XX, YY, ZZ, norm=LogNorm(), cmap="coolwarm",
                    vmin=1e-3, vmax=1e3)
ar.set_title("Spline interpolation: 'minmax'")
plt.colorbar(ax=ar, mappable=img)

fig.tight_layout()

### Detector source weights

We use the same spline method to create a spline describing the sinDec dependence of a signal MC weighted to a specific astrophysical flux modell (usually unbroken power law).

Depending on the src position, we expect more or less signal from that src.
This is equivalent to folding with the detector exposure function.

Our stacking form is described by a multi position search where the signal term gets modified to:

$$
    S^\text{tot} = \sum_{j=1}^{N_\text{srcs}} w_j S_{ij} \quad\text{with}\quad
    \sum_j w_j = 1 \quad\text{with}\quad w_j = w_j^\text{theo}\cdot w_j^\text{det}
$$

The weights are a combination of the exposure weights and a-priori fixed intrinsic source weights, eg. from a known gamma flux.

In [ ]:
# Small hack to change the gamma without recreating the grbllh object
gamma_override = 2.13

grbllh.energy_pdf_args["gamma"] = gamma_override
mc_sin_dec = np.sin(mc["dec"])
mc_bins = grbllh.energy_pdf_args["bins"][0]
mc_dict = {"trueE": mc["trueE"], "ow": mc["ow"]}

grbllh._spatial_signal_spl = grbllh._create_sin_dec_spline(
    sin_dec=mc_sin_dec, bins=mc_bins, mc=mc_dict)

sin_dec = np.linspace(-1.05, 1.05, 200)
y = np.exp(grbllh._spatial_signal_spl(sin_dec))

# MC needs proper weighting
gamma = grbllh.energy_pdf_args["gamma"]
mc_w = mc["ow"] * mc["trueE"]**(-gamma)
mc_bins = energy_pdf_args["bins"][0]
h, b = np.histogram(np.sin(mc["dec"]), bins=mc_bins, weights=mc_w, normed=True)

# Smooth it, charge it, odd it, quick truncate it
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.savgol_filter.html
m = get_binmids([b])[0]
redux = 10  # Window len is nearest odd number to (number of bins / redux)
window_len = int(2 * np.floor((len(b) / redux) / 2) + 1)  # Must be odd
_h = scsignal.savgol_filter(h, window_len, 3, mode="mirror")
plt.hist(m, bins=b, weights=_h, histtype="step", lw=2, color="C1", ls="--")
plt.hist(m, bins=b, weights=h, histtype="step", lw=2, color="C3")

# Plot spline (fitted to unsmoothed)
plt.plot(sin_dec, y, lw=2, color="C2")

# Get weights for some srcs
src_sin_dec = np.linspace(-1, 1, 11)
src_dec = np.arcsin(src_sin_dec)
src_w_theo = np.ones_like(src_dec)
w = grbllh.get_src_weights(src_dec=src_dec, src_w_theo=src_w_theo)

# Revoke norm for plotting to see if weights are on the curve
src_dec_w = np.exp(grbllh._spatial_signal_spl(src_sin_dec))
_w = w * np.sum(src_dec_w * src_w_theo)

plt.plot(src_sin_dec, _w, "wo", ms=7, mew=1.5, mec="k")
plt.vlines(np.sin(src_dec), 0, 1.05 * np.amax(y), colors="C7",
           lw=1, linestyles="--")

plt.xlabel("sin(dec)")
plt.ylabel("PDF")
plt.title("$\gamma = {:.1f}$".format(gamma))

plt.xlim(-1, 1)
plt.ylim(0, 1.05 * np.amax(y))
plt.tight_layout()

print(w)
print(w.sum())

Here we just use some ascending theoretical weights in both directions.

- w1 should resemble the sindec curve from above
- w2 should rise overall to the right (less steep or reversed from w1)
- w3 should fall overall to the right (steeper than w1)

In [ ]:
_src_dec = np.arcsin(np.linspace(-1, 1, 21))

# Compare for different theoretical weights
src_w_theo = np.ones_like(_src_dec)
w1 = grbllh.get_src_weights(src_dec=_src_dec, src_w_theo=src_w_theo)

src_w_theo = np.arange(len(_src_dec)) + 1
w2 = grbllh.get_src_weights(src_dec=_src_dec, src_w_theo=src_w_theo)

src_w_theo = (np.arange(len(_src_dec)) + 1)[::-1]
w3 = grbllh.get_src_weights(src_dec=_src_dec, src_w_theo=src_w_theo)

plt.plot(np.sin(_src_dec), w1, "o", label="theo = 1 (orig)")
plt.plot(np.sin(_src_dec), w2, "o", label="theo = arange")
plt.plot(np.sin(_src_dec), w3, "o", label="theo = arange[::-1]")
plt.legend(loc="upper left")

Check if weights are the same when using event density (skylab style) instead.
It should make no difference because the weights are normalized anyway.

**This will be included, if we get to fitting multiple years.
We can then use the same weights for the stacking and for normalizing ns per year.**

Note: The normalization and the pivot will not be included in the end, because they are constant for every source and for every dataset/year, so they get normalized out anyway.

In [ ]:
# Weight with numu diffuse 6yr flux norm, but same index as above
index = gamma_override
norm = 0.9 * 1e-18  # (GeV s sr cm^2)^-1, valid at 100 TeV = 1e5 GeV
pivot = 1e5
flux = norm * (mc["trueE"] / pivot)**(-index)
mc_w = mc["ow"] * flux * livetime * secinday
mc_bins = energy_pdf_args["bins"][0]

density = True
h, b = np.histogram(np.sin(mc["dec"]), bins=mc_bins, weights=mc_w,
                    density=density)

# Normalize (same as density=True)
if not density:
    h /= np.diff(b) * np.sum(h)

# PDF * Number of total events = Event densitiy
_h = h * mc_w.sum()  

mids = get_binmids([mc_bins])[0]
_ = plt.hist(mids, bins=mc_bins, weights=_h)

plt.xlabel("sindec")
plt.ylabel("Event density Nevts / sindec")

# Total events by integrating _h: Ntot = sum_i (_h_i * diff(bins)_i)
plt.title("Gamma = {:.2f}: Ntot = {:.2f}".format(
        index, np.sum(_h * np.diff(mc_bins))))

# Quick check the weight, no spline involved, directly use hist vals, so the
# Weights are not 100% percent equal. But equal enough to confirm
# Hack to include the first data point, as digitize is exclusive...
src_sin_dec[0] = -0.99
idx = np.digitize(src_sin_dec, bins=mc_bins, right=True) - 1
src_w_dec = _h[idx]
src_w_theo = np.ones_like(src_dec)
src_w = src_w_dec * src_w_theo / np.sum(src_w_dec * src_w_theo)

plt.plot(src_sin_dec, src_w_dec, "wo", ms=7, mew=1.5, mec="k")
plt.tight_layout()

print("Spline from PDF only")
print(w)

print("\nSpline from event density with livetime")
print(src_w.reshape(len(src_w), 1))

print("\nRatio")
print(w / src_w.reshape(len(src_w), 1))

### ln-LLH ratio

Plot llh and gradient.
The gradient is calculated analytically.
With this test, we simply want to check, if the gradient is OK and the likelihood behaves correctly.

#### Single Source

First with only one source.

Note: We test here "super-signal-like" events. Every event is exactly at the src position and every time is exactly in the rime window, where the ration is max- Only the energy is distributed as background.
So only for really large time windows (really large) which have insanely high background rates we drop lower than the injected ns in our prediction.
This is because the background term can only counter background-like events, which have a low signal over background ratio.
For the events injected here, this rate is super high, so we always "fit" the exact amount of injected events.

For such a setup for each event SoB is equal.
So the gradient is zero at:

\begin{align}
    0 &= -1 + \sum_i \frac{S}{n_b B}\cdot \frac{1}{n_s \frac{S}{n_b B} + 1}
       = -1 + N \frac{S}{n_b B}\cdot \frac{1}{n_s \frac{S}{n_b B} + 1} \\
    \Leftrightarrow \frac{1}{N} &= \frac{1}{n_s + \frac{n_b B}{S}} \\
    \Leftrightarrow N &= n_s + \frac{n_b B}{S}
\end{align}

So now if the signal is super large (and that's what we ensured by using our super-signal-like events) the term $\frac{n_b B}{S} \rightarrow 0$ and we get $\hat{n}_S = N$ which is exactly what we observe.

Only if we set super high $n_B$, our $\hat{n}_S$ shrinks as $\frac{n_b B}{S}$ gets larger and larger and in the end $\hat{n}_S$ even turns negative, when the 1 / SoB ratio is larger than N.

In [ ]:
# Snippet to plot and gradient
def plot_llh(ns, lnllh, lnllh_grad, ns_max, xmin, xmax):
    fig, (al, ar) = plt.subplots(1, 2, figsize=(10, 4))
    al.plot(ns, lnllh)
    if ns_max == 0:
        al.set_xlim(-1, 1)
    else:
        al.set_xlim(xmin, xmax)
    al.set_ylim(0, 1.05 * np.amax(lnllh))
    al.axvline(ns_max, 0, 1, ls="--", lw=2, color="C7")
    al.set_title("LLH")

    ar.plot(ns, lnllh_grad)
    ar.axhline(0, 0, 1, ls="--", lw=2, color="C7")
    ar.axvline(ns_max, 0, 1, ls="--", lw=2, color="C7")
    if ns_max == 0:
        al.set_xlim(-1, 1)
    else:
        al.set_xlim(xmin, xmax)
    ar.set_ylim(-5, 5)
    ar.set_title("LLH gradient in ns")
    fig.tight_layout()
    return fig, (al, ar)

In [ ]:
# Make up some setup
nsrcs = 1
src_t = np.random.choice(_exp["timeMJD"], size=nsrcs)
dt = np.array([-20, 200])

# Expected background with rate 5mHz, kind of realistic.
# Increase nb scale to see ns best fit shrink.
scale = 1e5
nb = 0.005 * np.diff(dt) * scale
src_ra = np.deg2rad([180])  # Arbitrarily placed single source
src_dec = np.deg2rad([10])
src_w_theo = np.ones_like(src_dec)

# Setup src record array
srcs = np.vstack((src_t, [dt[0]], [dt[1]],
                  [src_ra], [src_dec], src_w_theo))
names = ["t", "dt0", "dt1", "ra", "dec", "w_theo"]
srcs = np.core.records.fromarrays(srcs, names=names,
                                       formats=len(names) * ["float64"])
args = {"nb": nb, "srcs": srcs}

# Set the events artificially where the srcs are in space and nicely spaced
# times inside the search window, where time sob is large. Otherwise the llh
# is almost always peaked at 0
N = 10
mint, maxt = src_t + dt / secinday  # In MJD
timeMJD = np.linspace(mint, maxt, N)
X = np.random.choice(_exp, size=N)  # Only to copy the recarray structure
X["timeMJD"] = timeMJD
X["ra"] = np.ones_like(timeMJD) * src_ra
X["sinDec"] = np.ones_like(timeMJD) * np.sin(src_dec)
X["sigma"] = np.deg2rad(np.ones_like(timeMJD))

# Scan a single LLH for the chosen data above
n_ns = 500
xmin, xmax = 0, 2 * N
ns = np.linspace(xmin, xmax, n_ns)
lnllh = np.empty(n_ns)
lnllh_grad = np.empty(n_ns)
for i in range(n_ns):
    theta = {"ns": ns[i]}
    lnllh[i], lnllh_grad[i] = grbllh.lnllh_ratio(X, theta, args)

# Manual "fit" by scanning the maximum
ns_max = ns[np.argmax(lnllh)]

plot_llh(ns, lnllh, lnllh_grad, ns_max, xmin, xmax)
plt.show()

#### Multiple Sources -- All at same position

This time we use multiple sources, but all at the exact same location and with the exact same properties.
We expect the very same result as in the single source case above, because the weighted sum of the signal terms reduces to

\begin{align}
    S^\text{tot} &= \sum_{j=1}^{N_\text{srcs}} w_j S_{ij}
                 = S_{i} \sum_{j=1}^{N_\text{srcs}} \frac{1}{N}
                 = S_i \\
    \Lambda &= -2\ln\left(\frac{\mathcal{L}_0}{\mathcal{L}_1}\right)
             = -n_S + \sum_{i=1}^N\ln\left(\frac{n_S S^\text{tot}}{\langle n_B\rangle B_i} + 1\right)
             = -n_S + \sum_{i=1}^N\ln\left(\frac{n_S S_i}{\langle n_B\rangle B_i} + 1\right)
\end{align}

as all signal terms are exaxtly the same and no further background locations are introduced.

In [ ]:
# Repeat sources exactly as the single one from above
nsrcs = 5
_src_t = np.repeat(src_t, repeats=nsrcs, axis=0)
_dt = np.repeat(dt.reshape(1, 2), axis=0, repeats=nsrcs)
# Attention here: 100% overlapping windows so total BG is unchanged. To work
# in the stacking framework, we just split the expectation equally

# Increase nb scale to see ns best fit shrink
scale = 1e5
_nb = 0.005 * np.diff(_dt, axis=1).flatten() / nsrcs  * scale

_src_ra = np.repeat(src_ra, repeats=nsrcs, axis=0)
_src_dec = np.repeat(src_dec, repeats=nsrcs, axis=0)
_src_w_theo = np.ones_like(_src_dec)

# Setup src record array
srcs = np.vstack((_src_t, _dt[:, 0], _dt[:, 1],
                  _src_ra, _src_dec, _src_w_theo))

_srcs = np.core.records.fromarrays(srcs, names=names,
                                       formats=len(names) * ["float64"])
_args = {"nb": _nb, "srcs": _srcs}

# Also use the very same events for all sources here
_X = np.copy(X)

# Scan a single LLH for the chosen data above
n_ns = 500
xmin, xmax = 0, 2 * N
ns = np.linspace(xmin, xmax, n_ns)
_lnllh = np.empty(n_ns)
_lnllh_grad = np.empty(n_ns)
for i in range(n_ns):
    theta = {"ns": ns[i]}
    _lnllh[i], _lnllh_grad[i] = grbllh.lnllh_ratio(_X, theta, _args)

# Manual "fit" by scanning the maximum
_ns_max = ns[np.argmax(_lnllh)]

plot_llh(ns, _lnllh, _lnllh_grad, _ns_max, xmin, xmax)
plt.show()

#### Multiple Sources -- Different Right-Ascensions

Now the almost same thing, but with changed right ascensions only.
We distribute them equally around a fixed declination.
Also everything else is left as before.

This case is a bit more tricky, but a combination of the two cases above makes sure we still fit N events.

We inject the same number of events (N) but we get nsrcs times the BG (because the windows don't overlap anymore).
Each event only contributes to the window where it spatially is placed, so per source only N / nsrcs events (we choosed them so the number distribute nicely) have a SoB > 0.

This means, that the total signal term is reduced by a factor of nsrcs, as the zero signal terms can't compensate the unaffected backgound which is still the same for all events.

So even though our stacked signal term is reduced by a factor of nsrcs  we still get the same fit result, because the signal term is still huge and we still satisfy the condition $\frac{n_b B}{S}\rightarrow 0$.

But we need slightly less cranked up background rate to let the best fit ns shrink as in the previous cases.

In [ ]:
# Repeat sources exactly as the single one from above
nsrcs = 5
_src_t = np.repeat(src_t, repeats=nsrcs, axis=0)
_dt = np.repeat(dt.reshape(1, 2), axis=0, repeats=nsrcs)

# Windows don't overlap anymore, so use full BG for each window
# Increase nb scale to see ns best fit shrink
scale = 1e5
_nb = 0.005 * np.diff(_dt) * scale

# Handpick to let windows not overlap
_src_ra = np.deg2rad([0, 30, 60, 90, 120])
_src_dec = np.repeat(src_dec, repeats=nsrcs, axis=0)
_src_w_theo = np.ones_like(_src_dec)

# Setup src record array
srcs = np.vstack((_src_t, _dt[:, 0], _dt[:, 1],
                  _src_ra, _src_dec, _src_w_theo))

_srcs = np.core.records.fromarrays(srcs, names=names,
                                       formats=len(names) * ["float64"])
_args = {"nb": _nb, "srcs": _srcs}

# We used 5 srcs and 10 events, so we just repeat the ras once
# This is not very obvious on how to scale to arbirary Ns and nsrcs
# I'm not very sure here, how many events to inject to exactly match the cases
# above.
# Here we just have 2 evts per window and still have ns of 10, even though
# signal should get donwweighted to 1/5 of the two cases above per source.
_X = np.copy(X)
_X["ra"] = np.repeat(_src_ra, repeats=2)

# Scan a single LLH for the chosen data above
n_ns = 500
xmin, xmax = 0, 2 * N
ns = np.linspace(xmin, xmax, n_ns)
_lnllh = np.empty(n_ns)
_lnllh_grad = np.empty(n_ns)
for i in range(n_ns):
    theta = {"ns": ns[i]}
    _lnllh[i], _lnllh_grad[i] = grbllh.lnllh_ratio(_X, theta, _args)

# Manual "fit" by scanning the maximum
_ns_max = ns[np.argmax(_lnllh)]

plot_llh(ns, _lnllh, _lnllh_grad, _ns_max, xmin, xmax)
plt.show()

## Analysis

The analysis module grabs all the stuf from before and creates trial calculation from it.
So we test here, if everything wrapped up correctly and if we get OK looking test statistics from our trials.

A genreal note on how our experimental is handled:

Before we start out analysis, we split our data in off-time and on-time data.
On-time data is data around a a-priori fixed time frame around our sources we want to test.
We exclude this data until the very end, because we don't want to bias ourselfes as there is the possibility that the signal we want to find is in that on-time data.

The off-time data is everything else and is assumed to not contain the sought after signal.
The on-time time frame should be choosen large enough to account for that.
It should definitely be larger than the time frames we test for in our analysis.

### Fit LLH paramters

We test the same cases as with the bare LLH from above.
Everything should be the same.
This is basically a test to see if we wrapped the LLH correctly in the analysis module and if we get the same result from the fitter as from the manual scan in ns.

In [ ]:
# Snippet to plot and gradient
def plot_llh(ns, lnllh, lnllh_grad, ns_max, xmin, xmax):
    fig, (al, ar) = plt.subplots(1, 2, figsize=(10, 4))
    al.plot(ns, lnllh)
    if ns_max == 0:
        al.set_xlim(-1, 1)
    else:
        al.set_xlim(xmin, xmax)
    al.set_ylim(0, 1.05 * np.amax(lnllh))
    al.axvline(ns_max, 0, 1, ls="--", lw=2, color="C7")
    al.set_title("LLH. ns_max = {:.2f}".format(ns_max))

    ar.plot(ns, lnllh_grad)
    ar.axhline(0, 0, 1, ls="--", lw=2, color="C7")
    ar.axvline(ns_max, 0, 1, ls="--", lw=2, color="C7")
    if ns_max == 0:
        al.set_xlim(-1, 1)
    else:
        al.set_xlim(xmin, xmax)
    ar.set_ylim(-5, 5)
    ar.set_title("LLH gradient in ns")
    fig.tight_layout()
    return fig, (al, ar)

In [ ]:
# Create a grbllh likelihood object we want to test with
sin_dec_bins = np.linspace(-1, 1, 50)

min_logE = 1  #  min(np.amin(_exp["logE"]), np.amin(mc["logE"]))
max_logE = 10 #  max(np.amax(_exp["logE"]), np.amax(mc["logE"]))
logE_bins = np.linspace(min_logE, max_logE, 40)

spatial_pdf_args = {"bins": sin_dec_bins, "k": 3, "kent": True}

energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "col", "interpol_log": False}

time_pdf_args = {"nsig": 4., "sigma_t_min": 2., "sigma_t_max": 30.}

grbllh = LLH.GRBLLH(X=_exp, MC=mc, srcs=None,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

#### Single source

First we test if the module simply wrapps the LLH module correctly.
This should reproduce same results (not regarding random fluctuations of course) as in the section ln-llh ratio, as we test the same setup as above here.

In [ ]:
# Make up some setup
nsrcs = 1
src_t = np.random.choice(_exp["timeMJD"], size=nsrcs)
dt = np.array([-20, 200])

# Expected background with rate 5mHz, kind of realistic.
# Increase nb scale to see ns best fit shrink.
scale = 1e5
nb = 0.005 * np.diff(dt) * scale
src_ra = np.deg2rad([180])  # Arbitrarily placed single source
src_dec = np.deg2rad([10])
src_w_theo = np.ones_like(src_dec)

# Setup src record array
srcs = np.vstack((src_t, [dt[0]], [dt[1]],
                  [src_ra], [src_dec], src_w_theo))
names = ["t", "dt0", "dt1", "ra", "dec", "w_theo"]
srcs = np.core.records.fromarrays(srcs, names=names,
                                       formats=len(names) * ["float64"])
args = {"nb": nb, "srcs": srcs}

# Build the analysis module
ana = Analysis.TransientsAnalysis(srcs=srcs, llh=grbllh)

# Set the events artificially where the srcs are in space and nicely spaced
# times inside the search window, where time sob is large. Otherwise the llh
# is almost always peaked at 0
N = 10
mint, maxt = src_t + dt / secinday  # In MJD
timeMJD = np.linspace(mint, maxt, N)
X = np.random.choice(_exp, size=N)  # Only to copy the recarray structure
X["timeMJD"] = timeMJD
X["ra"] = np.ones_like(timeMJD) * src_ra
X["sinDec"] = np.ones_like(timeMJD) * np.sin(src_dec)
X["sigma"] = np.deg2rad(np.ones_like(timeMJD))

# Scan a single LLH for the chosen data above
n_ns = 500
xmin, xmax = 0, 2 * N
ns = np.linspace(xmin, xmax, n_ns)
lnllh = np.empty(n_ns)
lnllh_grad = np.empty(n_ns)
for i in range(n_ns):
    theta = {"ns": ns[i]}
    lnllh[i], lnllh_grad[i] = ana.llh.lnllh_ratio(X, theta, args)

# Manual "fit" by scanning the maximum
ns_max = ns[np.argmax(lnllh)]

plot_llh(ns, lnllh, lnllh_grad, ns_max, xmin, xmax)
plt.show()

In [ ]:
# Also let's quickly see, how the times are distributed within the time PDF
inj_trange = ana.llh.get_injection_trange(src_t=srcs["t"], dt=dt)
inj_trange = src_t + inj_trange.flatten() / secinday

x = np.linspace(inj_trange[0], inj_trange[1], 100)
y = ana.llh._soverb_time(t=x, src_t=srcs["t"], dt=dt)

plt.plot(x, y.reshape(len(x)))
plt.vlines(X["timeMJD"], 0, np.amax(y), colors="C7", linestyles="--", lw=2)
plt.show()

Test if we can do the same as above, but using the scipy fitter this time to get the maximum.
The LLH curve and the maximum should be identical to the ones above (except for small errors in scanning vs fitting ns).

In [ ]:
# Seed for the fitter
theta0 = {"ns": 1}

# Args must only contain "nb", everything else is used from self.srcs
args = {"nb": nb}

res = ana.fit_lnllh_ratio_params(X, theta0, args)

plot_llh(ns, lnllh, lnllh_grad, res.x[0], xmin, xmax)
plt.show()

#### Multiple Sources -- All at same position

In [ ]:
# Repeat sources exactly as the single one from above
nsrcs = 5
_src_t = np.repeat(src_t, repeats=nsrcs, axis=0)
_dt = np.repeat(dt.reshape(1, 2), axis=0, repeats=nsrcs)
# Attention here: 100% overlapping windows so total BG is unchanged. To work
# in the stacking framework, we just split the expectation equally

# Increase nb scale to see ns best fit shrink
scale = 1e5
_nb = 0.005 * np.diff(_dt, axis=1).flatten() / nsrcs  * scale

_src_ra = np.repeat(src_ra, repeats=nsrcs, axis=0)
_src_dec = np.repeat(src_dec, repeats=nsrcs, axis=0)
_src_w_theo = np.ones_like(_src_dec)

# Setup src record array
srcs = np.vstack((_src_t, _dt[:, 0], _dt[:, 1],
                  _src_ra, _src_dec, _src_w_theo))

_srcs = np.core.records.fromarrays(srcs, names=names,
                                       formats=len(names) * ["float64"])
_args = {"nb": _nb, "srcs": _srcs}

# Build the analysis module
ana = Analysis.TransientsAnalysis(srcs=_srcs, llh=grbllh)

# Also use the very same events for all sources here
_X = np.copy(X)

# Scan a single LLH for the chosen data above
n_ns = 500
xmin, xmax = 0, 2 * N
ns = np.linspace(xmin, xmax, n_ns)
_lnllh = np.empty(n_ns)
_lnllh_grad = np.empty(n_ns)
for i in range(n_ns):
    theta = {"ns": ns[i]}
    _lnllh[i], _lnllh_grad[i] = ana.llh.lnllh_ratio(_X, theta, _args)

# Manual "fit" by scanning the maximum
_ns_max = ns[np.argmax(_lnllh)]

plot_llh(ns, _lnllh, _lnllh_grad, _ns_max, xmin, xmax)
plt.show()

Again check using the class function

In [ ]:
# Seed for the fitter
theta0 = {"ns": 1}
# Args must only contain "nb", everything else is used from self.srcs
_args = {"nb": _nb}
    
res = ana.fit_lnllh_ratio_params(_X, theta0, _args, bounds=None)
    
plot_llh(ns, _lnllh, _lnllh_grad, res.x[0], xmin, xmax)
plt.show()

#### Multiple Sources -- Different Right-Ascencions

In [ ]:
# Repeat sources exactly as the single one from above
nsrcs = 5
_src_t = np.repeat(src_t, repeats=nsrcs, axis=0)
_dt = np.repeat(dt.reshape(1, 2), axis=0, repeats=nsrcs)

# Windows don't overlap anymore, so use full BG for each window
# Increase nb scale to see ns best fit shrink
scale = 1e5
_nb = 0.005 * np.diff(_dt) * scale

# Handpick to let windows not overlap
_src_ra = np.deg2rad([0, 30, 60, 90, 120])
_src_dec = np.repeat(src_dec, repeats=nsrcs, axis=0)
_src_w_theo = np.ones_like(_src_dec)

# Setup src record array
srcs = np.vstack((_src_t, _dt[:, 0], _dt[:, 1],
                  _src_ra, _src_dec, _src_w_theo))

_srcs = np.core.records.fromarrays(srcs, names=names,
                                       formats=len(names) * ["float64"])
_args = {"nb": _nb, "srcs": _srcs}

# Build the analysis module
ana = Analysis.TransientsAnalysis(srcs=_srcs, llh=grbllh)

# We used 5 srcs and 10 events, so we just repeat the ras once
# This is not very obvious on how to scale to arbirary Ns and nsrcs
# I'm not very sure here, how many events to inject to exactly match the cases
# above.
# Here we just have 2 evts per window and still have ns of 10, even though
# signal should get donwweighted to 1/5 of the two cases above per source.
_X = np.copy(X)
_X["ra"] = np.repeat(_src_ra, repeats=2)

# Scan a single LLH for the chosen data above
n_ns = 500
xmin, xmax = 0, 2 * N
ns = np.linspace(xmin, xmax, n_ns)
_lnllh = np.empty(n_ns)
_lnllh_grad = np.empty(n_ns)
for i in range(n_ns):
    theta = {"ns": ns[i]}
    _lnllh[i], _lnllh_grad[i] = grbllh.lnllh_ratio(_X, theta, _args)

# Manual "fit" by scanning the maximum
_ns_max = ns[np.argmax(_lnllh)]

plot_llh(ns, _lnllh, _lnllh_grad, _ns_max, xmin, xmax)
plt.show()

Again check with the class function

In [ ]:
# Seed for the fitter
theta0 = {"ns": 1}
# Args must only contain "nb", everything else is used from self.srcs
_args = {"nb": _nb}
    
res = ana.fit_lnllh_ratio_params(_X, theta0, _args, bounds=None)
    
plot_llh(ns, _lnllh, _lnllh_grad, res.x[0], xmin, xmax)
plt.show()

### Trials

Using all the modules from above we can run trials with pure background now.

A trial is a single pseudo experiment we perform and evaluate to get an idea of the underlying statistical distribtuion and to build our test statistic from which we can infer the significance of real data later.
For that we need to generate sets of pseudo-data which has background-like properties.
The `bg_injector`, `bg_rate_injector` and `rate_function` classes are used to generate these properties.

So each trial consist of the following steps, in which the source positions are always fixed and a-priori known:

1. Determine the expected number background events per source time window we test.
   This is derived from the `bg_rate_injector` which returns a list of sampled times for each time window.
   It knows the expected rate from the given `rate_function`
2. In addition to our sampled times, we need all the other event features we have on real data (positions, energy, uncertainty) , because our sampled pseudo-data should have the same properties as real data.
   These missing properties are generated by the `bg_injector` class.
3. When we sampled our pseudo-events we need to fit the LLH to this set of events and see what best fit we get.
4. We do that a lot of times and see how our best fits are distributed which gives us a so called test statistic which describes the distribution of LLH firs using BG only.

On background-like events we expect to get a null fit result most of the times, because no signal is present.
But out of chance, we sometimes get a combination of background-like events, that has very signal-like properties.
The so build test statistic is then used to see how unlikely the single fit to our on-time data was and how lucky we'd have to get to observe that result out of chance from pure background.

#### BG only

In [ ]:
# Create a bg rat einjector model
def filter_runs(run):
    """
    Filter runs as stated in jfeintzig's doc.
    """
    exclude_runs = [120028, 120029, 120030, 120087, 120156, 120157]
    if ((run["good_i3"] == True) & (run["good_it"] == True) &
        (run["run"] not in exclude_runs)):
        return True
    else:
        return False

# Let's create an injector using a goodrun list.
runlist="data/runlists/ic86-i-goodrunlist.json"
rate_func = RateFunc.Sinus1yrRateFunction()
runlist_inj = BGRateInj.RunlistBGRateInjector(runlist, filter_runs, rate_func)

# `fit` the injector to make it usable
times = exp["timeMJD"]
rate_func = runlist_inj.fit(T=times, x0=None, remove_zero_runs=True)

In [ ]:
# Create our bg injector, here we use the data resampler
data_inj = BGInj.DataBGInjector()
data_inj.fit(sample)

In [ ]:
# Least we need a LLH, use the GRBLLH
sin_dec_bins = np.linspace(-1, 1, 50)

# Choose borders and bins by eye
min_logE = 1 
max_logE = 10
logE_bins = np.linspace(min_logE, max_logE, 40)

spatial_pdf_args = {"bins": sin_dec_bins, "k": 3, "kent": True}
energy_pdf_args = {"bins": [sin_dec_bins, logE_bins],
                   "gamma": 2., "fillval": "col", "interpol_log": False}
time_pdf_args = {"nsig": 4.}

grbllh = LLH.GRBLLH(X=_exp, MC=mc,
                    spatial_pdf_args=spatial_pdf_args,
                    energy_pdf_args=energy_pdf_args,
                    time_pdf_args=time_pdf_args)

In [ ]:
# Create some srcs to test for
dt = np.atleast_2d([[-20, 200], [-5, 20], [0, 300]])
n_srcs = len(dt)

dtype = [("t", np.float), ("dt0", np.float), ("dt1", np.float),
         ("ra", np.float), ("dec", np.float)]
srcs = np.zeros((n_srcs, ), dtype=dtype)
srcs["t"] = np.random.choice(_exp["timeMJD"], size=n_srcs)
srcs["dt0"] = dt[:, 0]
srcs["dt1"] = dt[:, 1]
srcs["ra"] = np.random.uniform(0, 2 * np.pi, n_srcs)
srcs["dec"] = np.arcsin(np.random.uniform(-1, 1, n_srcs))

In [ ]:
# And now the analysis object
ana = Analysis.TransientsAnalysis(srcs=srcs, llh=grbllh)

In [ ]:
self = ana

_X = []
times = []
rnd_ra = []
args = []

for src_idx in range(len(srcs)):
    # Samples times and thus number of bg expectated events
    t = self.srcs["t"][src_idx]
    dt = [self.srcs["dt0"][src_idx], self.srcs["dt1"][src_idx]]
    _times = runlist_inj.sample(t=t, trange=dt, ntrials=1)[0]
    nb = len(_times)
    times.append(_times)
    args.append({"nb": nb})
    # Sample rest of features
    if nb > 0:
        _X.append(data_inj.sample(n_samples=nb))
        rnd_ra.append(np.random.uniform(0, 2. * np.pi, size=nb))

names = ["ra", "sinDec", "timeMJD", "logE", "sigma"]
dtype = [(n, t) for (n, t) in zip(names, len(names) * [np.float])]
nb_tot = np.sum([d["nb"] for d in args])
X = np.empty((nb_tot, ), dtype=dtype)
# Make output array in compatible format
_X = flatten_list_of_1darrays(_X)
X["ra"] = flatten_list_of_1darrays(rnd_ra)
X["sinDec"] = np.sin(_X[:, 1])
X["logE"] = _X[:, 0]
X["sigma"] = _X[:, 2]
X["timeMJD"] = flatten_list_of_1darrays(times)

nb_tot

In [ ]:
ana.fit_lnllh_ratio_params(X, theta0={"ns": 1}, args=args)